# Transformación

In [48]:
import pandas as pd
import numpy as np
import os 
import glob

In [ ]:
MAPA_TR = {
    "01_VOLTAJE_AC_DEL_SISTEMA_L1_L2": "voltaje_ac_l1_l2",
    "02_VOLTAJE_AC_DEL_SISTEMA_L2-L3": "voltaje_ac_l2_l3",
    "03_VOLTAJE_AC_DEL_SISTEMA_L3-L1": "voltaje_ac_l3_l1",
    "11_CORRIENTE_AC_DE_LA_CARGA_L1": "corriente_ac_l1",
    "12_CORRIENTE_AC_DE_LA_CARGA_L2": "corriente_ac_l2",
    "13_CORRIENTE_AC_DE_LA_CARGA_L3": "corriente_ac_l3",
    "14_POTENCIA_ACTIVA_DE_LA_CARGA": "potencia_activa_kw",
    "15_POTENCIA_REACTIVA_DE_LA_CARGA": "potencia_reactiva_kvar",
    "16_POTENCIA_APARENTE_DE_LA_CARGA": "potencia_aparente_kva", 
    "17_FACTOR_DE_POTENCIA_DE_LA_CARGA": "factor_potencia"
}

MAPA_ML = {
    "ANALOG_INPUT_-_ML_CURRENT_AC_R": "corriente_ac_r",
    "ANALOG_INPUT_-_ML_CURRENT_AC_S": "corriente_ac_s",
    "ANALOG_INPUT_-_ML_CURRENT_AC_T": "corriente_ac_t",
    "ANALOG_INPUT_-_ML_VOLTAGE_AC_R-S": "voltaje_ac_rs",
    "ANALOG_INPUT_-_ML_VOLTAGE_AC_S-T": "voltaje_ac_st",
    "ANALOG_INPUT_-_ML_VOLTAGE_AC_T-R": "voltaje_ac_tr",
    "ANALOG_INPUT_C_SALA_S01": "temp_sala_s01",
    "ANALOG_INPUT_C_SALA_S02": "temp_sala_s02"
}

MAPA_RECT1 = {
    "01_-_VOLTAJE_AC_DEL_SISTEMA" : "voltaje_ac_vs",
    "02_-_VOLTAJE_DC_DEL_SISTEMA": "voltaje_dc_vs",
    "03_-_CORRIENTE_DC_DEL_SISTEMA": "corriente_dc_cs",
    "11_-_CORRIENTE_DC_DE_LA_CARGA": "corriente_dc_carga"
}

MAPA_RECT2 = {
    "01_-_VOLTAJE_AC_DEL_SISTEMA" : "voltaje_ac_vs",
    "02_-_VOLTAJE_DC_DEL_SISTEMA": "voltaje_dc_vs",
    "03_-_CORRIENTE_DC_DEL_SISTEMA": "corriente_dc_cs",
    "11_-_CORRIENTE_DC_DE_LA_CARGA": "corriente_dc_carga"
}

In [50]:
def limpiar_fecha(fecha_str):
    """
    TR - IDEO CALI,"01 - VOLTAJE AC DEL SISTEMA L1-L2","216.0","V","1/1/2025, 12:50:03 a.Â m."

    Arregla el formato y elimina caracteres de codificación corruptos (Â).
    Entrada: '1/1/2025, 12:00:00 a.Â m.' o '1/1/2025, 12:00:00 a.\xa0m.'
    Salida: '1/1/2025 12:00:00 AM'
    """

    if not isinstance(fecha_str, str):
        return str(fecha_str)
    

    limpia = fecha_str.replace("Â", "") \
                      .replace("\xa0", " ") \
                      .replace(",", "") 
    
    limpia = limpia.lower() \
                   .replace("a. m.", "AM") \
                   .replace("p. m.", "PM") \
                   .replace("a.m.", "AM") \
                   .replace("p.m.", "PM") \
                   .replace("am", "AM") \
                   .replace("pm", "PM")
    
    return limpia.strip() # Eliminar espacios 

In [51]:
def procesar_carpeta_sensor(ruta, nombre_carpeta, nombre_columna_db):
    
    ruta_busqueda = os.path.join(ruta, nombre_carpeta, "*.csv") 
    archivos = glob.glob(ruta_busqueda)
    
    if not archivos:
        print(f"ADVERTENCIA: No se encontraron archivos en {nombre_carpeta}")
        return None

    print(f"Procesando {nombre_carpeta} ({len(archivos)} archivos)...")
    
    dfs_meses = []
    
    for archivo in archivos:
        try:
            # Leemos el CSV. Importante: dtype=str para no romper la lectura con 'Unplugged'
            df_temp = pd.read_csv(archivo, encoding='latin-1', dtype=str) # Probamos latin-1 por los caracteres raros
            
            df_temp['Time_Clean'] = df_temp['Time'].apply(limpiar_fecha)
            df_temp['timestamp'] = pd.to_datetime(df_temp['Time_Clean'], format='%d/%m/%Y %I:%M:%S %p', errors='coerce')
            
            # Convertimos Value a números. Los textos se vuelven NaN.
            df_temp['valor_numerico'] = pd.to_numeric(df_temp['Value'], errors='coerce')
            
            #Eliminar filas con NaN
            df_temp = df_temp.dropna(subset=['valor_numerico', 'timestamp'])
            
            # Seleccionamos solo lo que sirve (ya limpio)
            df_util = df_temp[['timestamp', 'valor_numerico']].copy()
            
            dfs_meses.append(df_util)
            
        except Exception as e:
            print(f"Error leyendo {os.path.basename(archivo)}: {e}")

    if not dfs_meses:
        return None

    # Unir enero y diciembre 
    df_anual = pd.concat(dfs_meses, ignore_index=True)
    
    # Ordenar por fecha
    df_anual = df_anual.sort_values('timestamp')
    df_anual = df_anual.set_index('timestamp')
    

    # *************************RESAMPLING (Normalización de tiempo)*******************************************************************
    # Como los datos vienen a 12:01:59 y otros a 12:05:00, promediamos cada 10 minutos
    # Esto alinea los datos y reduce el ruido.
    df_resampled = df_anual.resample('10min').mean()
    
    # Renombramos la columna 'valor_numerico' al nombre real del sensor (ej: voltaje_ac_l1_l2)
    df_resampled.columns = [nombre_columna_db]
    
    return df_resampled

In [52]:
ELEMENTOS_DCE = [
    ("TR",    "./Datos/DCE_DATOS_2025/TR",    MAPA_TR),
    ("ML",    "./Datos/DCE_DATOS_2025/ML",    MAPA_ML),
    ("Rect1", "./Datos/DCE_DATOS_2025/Rect1", MAPA_RECT1),
    ("Rect2", "./Datos/DCE_DATOS_2025/Rect2", MAPA_RECT2)
]

for nombre_dev, ruta_base, mapa_sensores in ELEMENTOS_DCE:
    print(f"\n" + "="*50)
    print(f"PROCESANDO DISPOSITIVO: {nombre_dev}")
    print(f"="*50)
    
    dfs_del_dispositivo = []
    

    for carpeta_sensor, columna_db in mapa_sensores.items():            
        df_sensor = procesar_carpeta_sensor(ruta_base, carpeta_sensor, columna_db)
        
        if df_sensor is not None:
            dfs_del_dispositivo.append(df_sensor)
    
    if dfs_del_dispositivo:
        print(f"Unificando sensores de {nombre_dev}...")
        df_final = pd.concat(dfs_del_dispositivo, axis=1)
        
        # Limpieza final (interpolación o reset index)
        df_final = df_final.reset_index()
        
        # Guardar
        carpeta_destino = os.path.join("Datos", "Historico")
        os.makedirs(carpeta_destino, exist_ok=True)

        nombre_archivo = f"consolidado_{nombre_dev}_2025.csv"
        ruta_completa = os.path.join(carpeta_destino, nombre_archivo)

        df_final.to_csv(ruta_completa, index=False)

        print(f"Guardado {nombre_archivo} ({len(df_final)} filas)")
    else:
        print(f"ADVERTENCIA: No se generaron datos para {nombre_dev}")


PROCESANDO DISPOSITIVO: TR
Procesando 01_VOLTAJE_AC_DEL_SISTEMA_L1_L2 (12 archivos)...
Procesando 02_VOLTAJE_AC_DEL_SISTEMA_L2-L3 (12 archivos)...


KeyboardInterrupt: 

# Base de datos

In [ ]:
from sqlalchemy import create_engine, text
import getpass

In [ ]:
USER = 'root'
HOST = 'localhost'
PORT = '3306'
DB_NAME = 'NODO_IDEO'
PASS = getpass.getpass("contraseña de MySQL: ")

In [ ]:
# Cadena de conexión SIN base de datos (para poder crearla)
URL_SERVER = f"mysql+pymysql://{USER}:{PASS}@{HOST}:{PORT}"
# Cadena de conexión CON base de datos (para crear tablas)
URL_DB = f"{URL_SERVER}/{DB_NAME}"

# Conectar al servidor para crear la BD
engine_server = create_engine(URL_SERVER)

with engine_server.connect() as conn:
    # Crea la BD con utf8mb4
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {DB_NAME} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;"))
    print(f"Base de datos '{DB_NAME}' verificada.")


Base de datos 'NODO_IDEO' verificada.


In [ ]:
engine_db = create_engine(URL_DB)

with engine_db.connect() as conn:

    sql_tr = """
    CREATE TABLE IF NOT EXISTS tr_historico (
        timestamp DATETIME NOT NULL,
        voltaje_ac_l1_l2 FLOAT,
        voltaje_ac_l2_l3 FLOAT,
        voltaje_ac_l3_l1 FLOAT,
        corriente_ac_l1 FLOAT,
        corriente_ac_l2 FLOAT,
        corriente_ac_l3 FLOAT,
        potencia_activa_kw FLOAT,
        potencia_reactiva_kvar FLOAT,
        potencia_aparente_kva FLOAT,
        factor_potencia FLOAT,
        PRIMARY KEY (timestamp)
        ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """
    
    
    sql_ml = """
    CREATE TABLE IF NOT EXISTS ml_historico (
        timestamp DATETIME NOT NULL,
        corriente_ac_r FLOAT,
        corriente_ac_s FLOAT,
        corriente_ac_t FLOAT,
        voltaje_ac_rs FLOAT,
        voltaje_ac_st FLOAT,
        voltaje_ac_tr FLOAT,
        temp_sala_s01 FLOAT,
        temp_sala_s02 FLOAT,
        PRIMARY KEY (timestamp)
        ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """
    
    sql_rect1 = """
    CREATE TABLE IF NOT EXISTS rect1_historico (
        timestamp DATETIME NOT NULL,
        voltaje_ac_vs FLOAT,
        voltaje_dc_vs FLOAT,
        corriente_dc_cs FLOAT,
        corriente_dc_carga FLOAT,
        PRIMARY KEY (timestamp)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """

    sql_rect2 = """
    CREATE TABLE IF NOT EXISTS rect2_historico (
        timestamp DATETIME NOT NULL,
        voltaje_ac_vs FLOAT,
        voltaje_dc_vs FLOAT,
        corriente_dc_cs FLOAT,
        corriente_dc_carga FLOAT,
        PRIMARY KEY (timestamp)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """

    conn.execute(text(sql_tr))
    conn.execute(text(sql_ml))
    conn.execute(text(sql_rect1))
    conn.execute(text(sql_rect2))


# Carga 

In [ ]:
import pandas as pd
import os
from sqlalchemy import text

## DCE

In [ ]:
def cargar_datos_sql(engine_db):

    carpeta_origen = os.path.join("Datos", "Historico")

    mapa_carga = {
        "consolidado_TR_2025.csv":    "tr_historico",
        "consolidado_ML_2025.csv":    "ml_historico",
        "consolidado_Rect1_2025.csv": "rect1_historico",
        "consolidado_Rect2_2025.csv": "rect2_historico"
    }

    for archivo_csv, nombre_tabla in mapa_carga.items():
        ruta_completa = os.path.join(carpeta_origen, archivo_csv)
        
        if not os.path.exists(ruta_completa):
            print(f" AVISO: No se encontró {archivo_csv}, saltando...")
            continue

        try:
            df = pd.read_csv(ruta_completa)
            
            if 'timestamp' in df.columns:
                df['timestamp'] = pd.to_datetime(df['timestamp'])
            
            print(f" Subiendo {len(df)} registros a la tabla '{nombre_tabla}'...")
            
            # if_exists='append': Agrega los datos a la tabla que ya creaste.
            # index=False: No subir el índice de pandas (0, 1, 2...) como columna.
            # chunksize=1000: Sube de a 1000 filas para no saturar la red/memoria.
            df.to_sql(
                name=nombre_tabla,
                con=engine_db,
                if_exists='append', 
                index=False,
                chunksize=1000
            )
            
            print(f"Carga completada en '{nombre_tabla}'.")

        except Exception as e:
            print(f" ERROR cargando {nombre_tabla}: {e}")
            # Tip: Si el error es "Duplicate entry", significa que ya corriste esto antes.

    print("\n PROCESO DE CARGA FINALIZADO.")

In [ ]:
cargar_datos_sql(engine_db)

 Subiendo 52560 registros a la tabla 'tr_historico'...
Carga completada en 'tr_historico'.
 Subiendo 52560 registros a la tabla 'ml_historico'...
Carga completada en 'ml_historico'.
 Subiendo 52560 registros a la tabla 'rect1_historico'...
Carga completada en 'rect1_historico'.
 Subiendo 52560 registros a la tabla 'rect2_historico'...
Carga completada en 'rect2_historico'.

 PROCESO DE CARGA FINALIZADO.


## Datos Manuales

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

In [ ]:
engine_db = create_engine(URL_DB)

In [ ]:
archivo_excel = "Datos/datos_manuales.xlsx"

with engine_db.connect() as conn:
    
    # INFO_NODO
    conn.execute(text("DROP TABLE IF EXISTS info_nodo"))
    sql_info = """
    CREATE TABLE info_nodo (
        id INT AUTO_INCREMENT PRIMARY KEY,
        nombre_nodo VARCHAR(50),
        tipo VARCHAR(10),
        codigo VARCHAR(50),
        regional VARCHAR(50),
        direccion VARCHAR(100),
        capacidad_kva FLOAT,
        voltaje_sistema_dc FLOAT
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """
    conn.execute(text(sql_info))
    

    try:
        df = pd.read_excel(archivo_excel, sheet_name="info_nodo")
        df.to_sql('info_nodo', con=engine_db, if_exists='append', index=False)
        print("Informacion de nodo actualizada")
    except Exception as e:
        print(f"Error cargando datos: {e}")


    # INVENTARIO_AC
    conn.execute(text("DROP TABLE IF EXISTS inventario_ac"))
    
    sql_ac = """
    CREATE TABLE inventario_ac (
        id INT AUTO_INCREMENT PRIMARY KEY,
        ubicacion VARCHAR(50),
        componente VARCHAR(50),      
        marca VARCHAR(50),           
        referencia VARCHAR(100),     
        capacidad_amps FLOAT,        
        calibre_cable VARCHAR(50) 
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """
    conn.execute(text(sql_ac))
    
    try:
        df = pd.read_excel(archivo_excel, sheet_name="inventario_ac")
        df.to_sql('inventario_ac', con=engine_db, if_exists='append', index=False)
        print("inventario AC actualizada")
    except Exception as e:
        print(f"Error cargando datos: {e}")


    # INVENTARIO_DC_PDB
    conn.execute(text("DROP TABLE IF EXISTS inventario_dc_pdb"))
    
    sql_pdb = """
    CREATE TABLE inventario_dc_pdb (
        id INT AUTO_INCREMENT PRIMARY KEY,
        pdb_nombre VARCHAR(20),      
        fuente VARCHAR(10),          
        posicion INT,       
        estado VARCHAR(20),          
        capacidad FLOAT,
        corriente FLOAT,
        equipo_refencia VARCHAR(100) 
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """
    conn.execute(text(sql_pdb))
    
    try:
        df = pd.read_excel(archivo_excel, sheet_name="inventario_pdb")
        df.to_sql('inventario_dc_pdb', con=engine_db, if_exists='append', index=False)
        print("inventario PDB actualizada")
    except Exception as e:
        print(f"Error cargando datos: {e}")
        
    # Confirmar cambios de estructura
    conn.commit()


Informacion de nodo actualizada
inventario AC actualizada
inventario PDB actualizada


# API

In [ ]:
import requests
import urllib3  

# silencia la advertencia roja
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

response = requests.get("https://10.159.125.33", verify=False, timeout=5)
print(response) 

<Response [200]>


In [ ]:
import requests
import urllib3

# Deshabilitar advertencias SSL 
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

DCE_HOST = "https://10.159.125.33"  
ACCESS_TOKEN = "954ff0a0-4e9a-48b3-b124-321f86f923e3"   


url = f"{DCE_HOST}/isxg/v1/devices"

headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

print("Enviando GET a:", url)

response = requests.get(url, headers=headers, verify=False)

print("Código HTTP:", response.status_code)

if response.status_code == 200:
    print("\nRespuesta OK (primeros 500 caracteres):\n")
    print(response.text[:500])
else:
    print("\nError:")
    print(response.text)


🔵 Enviando GET a: https://10.159.125.33/isxg/v1/devices
Código HTTP: 200

✅ Respuesta OK (primeros 500 caracteres):

[{"severity":0,"severityText":"None","inMaintenance":false,"isxcGuid":"B7e755e_nbSNMPEncFA60E49D","model":"AP8461","type":"Rack PDU","hostname":"10.170.118.126","ipAddress":"10.170.118.126","privateSide":false,"label":"EPM-L25-PDU-DER","serialNumber":"5A1946E02998","location":"SALA 03 - RACK L25","supplemental":{"InputPhaseCount":3,"OutputPhaseCount":3}},{"severity":0,"severityText":"None","inMaintenance":false,"isxcGuid":"B7e755e_nbSNMPEnc88F4D655","model":"EATON - SC200","type":"DC Rectifier",


In [17]:
import requests
import urllib3
import getpass

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# --- TUS DATOS ---
IP = "10.159.125.33"
USUARIO = "mhoyosme"
PASSWORD = getpass.getpass("🔑 Ingresa tu contraseña una vez más: ")

# Lista de todas las posibles direcciones de Login de DCE
rutas_a_probar = [
    f"https://{IP}/api/v1/oauth/token",        # Opción Estándar Moderna
    f"https://{IP}/isxg/api/v1/oauth/token",   # Opción con prefijo ISXG
    f"https://{IP}/isxg/oauth/token",          # Opción versiones intermedias
    f"https://{IP}/oauth/token",               # Opción raíz
    f"https://{IP}/rest/oauth/token"           # Opción antigua
]

print(f"\n🔍 Buscando la puerta de entrada correcta para el usuario '{USUARIO}'...")

payload = {
    "username": USUARIO,
    "password": PASSWORD,
    "grant_type": "password"
}

url_ganadora = None

for url in rutas_a_probar:
    print(f"👉 Probando: {url} ...", end=" ")
    
    try:
        # Probamos enviar los datos como FORM (data=...) que es lo estándar en OAuth
        response = requests.post(url, data=payload, verify=False, timeout=5)
        
        if response.status_code == 200:
            print("✅ ¡ÉXITO! (200 OK)")
            print("   🎉 ¡Encontramos la URL correcta!")
            url_ganadora = url
            break # Dejamos de buscar
        elif response.status_code == 404:
            print("❌ (404 Not Found - Ruta equivocada)")
        elif response.status_code == 401:
            print("⛔ (401 Unauthorized - Ruta existe pero rechazó credenciales)")
            # A veces el 401 aquí significa que la ruta es correcta pero el formato no
            # Pero sigamos buscando por si acaso hay un 200 en otra.
        elif response.status_code == 405:
            print("🚫 (405 Method Not Allowed - Ruta equivocada)")
        else:
            print(f"⚠️ ({response.status_code})")
            
    except Exception as e:
        print(f"💀 Error de conexión: {e}")

print("-" * 50)
if url_ganadora:
    print(f"🏆 LA URL QUE DEBES USAR EN TU SCRIPT ES:\n")
    print(f'self.auth_url = "{url_ganadora}"')
    print("\nCopia esa línea y pégala en tu clase GestorDCE.")
else:
    print("❌ Ninguna ruta funcionó estándar. Posibles causas:")
    print("1. La contraseña se escribió mal en este intento.")
    print("2. El servidor requiere un encabezado 'Content-Type' específico.")


🔍 Buscando la puerta de entrada correcta para el usuario 'mhoyosme'...
👉 Probando: https://10.159.125.33/api/v1/oauth/token ... ❌ (404 Not Found - Ruta equivocada)
👉 Probando: https://10.159.125.33/isxg/api/v1/oauth/token ... ⛔ (401 Unauthorized - Ruta existe pero rechazó credenciales)
👉 Probando: https://10.159.125.33/isxg/oauth/token ... ✅ ¡ÉXITO! (200 OK)
   🎉 ¡Encontramos la URL correcta!
--------------------------------------------------
🏆 LA URL QUE DEBES USAR EN TU SCRIPT ES:

self.auth_url = "https://10.159.125.33/isxg/oauth/token"

Copia esa línea y pégala en tu clase GestorDCE.


In [ ]:
import requests
import urllib3
import pandas as pd
import re

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
IP = "10.159.125.33"
TOKEN = "e264daff-a662-410b-891d-514508c0fb3a" 
URL_DEVICES = f"https://{IP}/isxg/v1/devices"

def buscar_equipos_ideo():
    print(f"Buscando equipos en: {URL_DEVICES}...")
    
    headers = {
        "Authorization": f"Bearer {TOKEN}",
        "Accept": "application/json"
    }
    
    try:
        response = requests.get(URL_DEVICES, headers=headers, verify=False, timeout=10)
        
        if response.status_code == 200:
            todos_los_equipos = response.json()
            print(f"Total de equipos en el servidor: {len(todos_los_equipos)}")
            
            mis_equipos = []
            
            print(f"{'NOMBRE (Label)':<40} | {'MODELO':<25} | {'ID ÚNICO (isxcGuid)'}")
            print("-" * 100)
            
            for equipo in todos_los_equipos:
                label = equipo.get('label', 'Sin Nombre')
                
                label_up = label.upper()

                if re.search(r'\bIDEO\b', label_up) or re.search(r'\bCALI\b', label_up):
                    modelo = equipo.get('model', 'N/A')
                    guid = equipo.get('isxcGuid')

                    print(f"{label:<40} | {modelo:<25} | {guid}")

                    mis_equipos.append({
                        "Nombre": label,
                        "ID": guid,
                        "Modelo": modelo
                    })
                            
            if not mis_equipos:
                print("No encontré equipos con la palabra 'IDEO' o 'CALI'. Revisa cómo están nombrados.")
            
            return mis_equipos

        else:
            print(f"Error: {response.status_code}")
            
    except Exception as e:
        print(f"Error de conexión: {e}")


equipos = buscar_equipos_ideo()

Buscando equipos en: https://10.159.125.33/isxg/v1/devices...
Total de equipos en el servidor: 1337

EQUIPOS ENCONTRADOS (IDEO / CALI):
NOMBRE (Label)                           | MODELO                    | ID ÚNICO (isxcGuid)
----------------------------------------------------------------------------------------------------
RECT 02 - IDEO CALI - (10.170.203.7)     | ELTEK - SMARTPACK 2 TOUCH | B7e755e_nbSNMPEnc7D425FC9
TABLERO - IDEO CALI - (10.170.203.2)     | EATON - SC200             | B7e755e_nbSNMPEnc60BA612C
MG - IDEO CALI                           | COMAP - INTELILITE AMF25 - MRS - 3P | B7e755e_nbModbusEnc11703964
RECT 01 - IDEO CALI - (10.170.203.6)     | ELTEK - SMARTPACK 2 TOUCH | B7e755e_nbSNMPEncD1F128BD
TR - IDEO CALI                           | COMAP - IL-NT-AMF25 - 3P  | B7e755e_nbModbusEnc4B4D008D


In [11]:
import requests
import urllib3
import time
import getpass

# Desactivar alertas SSL
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class GestorDCE:
    def __init__(self, ip, usuario, password):
        self.base_url = f"https://{ip}/isxg/v1"       
        self.auth_url = f"https://{ip}/isxg/oauth/token"
        self.usuario = usuario
        self.password = password
        self.token = None
        self.token_time = 0
        self.TTL = 250  # 4 minutos de vida útil (renueva antes de los 5 min)

    def _obtener_nuevo_token(self):
        """Pide un token nuevo automáticamente"""
        print("Renovando Token de acceso...")
        
        # Datos para OAuth2
        payload = {
            "username": self.usuario,
            "password": self.password,
            "grant_type": "password"
        }
        
        try:
            resp = requests.post(self.auth_url, data=payload, verify=False, timeout=15)
            
            if resp.status_code == 200:
                data = resp.json()
                self.token = data['access_token']
                self.token_time = time.time()
                print("Token renovado exitosamente.")
            else:
                # Si falla, imprimimos el error crudo para depurar
                raise Exception(f"Error Login ({resp.status_code}): {resp.text}")
                
        except Exception as e:
            print(f"Error crítico obteniendo token: {e}")
            raise e

    def get_headers(self):
        """Devuelve las cabeceras con un token válido"""
        ahora = time.time()
        # Si no hay token o ya casi vence, pedimos uno nuevo
        if not self.token or (ahora - self.token_time > self.TTL):
            self._obtener_nuevo_token()
        
        return {
            "Authorization": f"Bearer {self.token}",
            "Accept": "application/json"
        }

    def consultar_equipo(self, device_id):
        """Consulta todos los sensores de un equipo"""
        endpoint = f"{self.base_url}/devices/{device_id}/sensors"
        
        try:
            headers = self.get_headers() # Aquí se hace el login automático si hace falta
            resp = requests.get(endpoint, headers=headers, verify=False, timeout=15)
            
            if resp.status_code == 200:
                return resp.json()
            else:
                print(f" Error consultando equipo {device_id}: {resp.status_code}")
                return None
        except Exception as e:
            print(f" Error de conexión consultando equipo: {e}")
            return None


# PRUEBA FINAL EN TIEMPO REAL
if __name__ == "__main__":
    IP = "10.159.125.33"
    USER = "mhoyosme"
    PASS = getpass.getpass(" Contraseña: ") 

    dce = GestorDCE(IP, USER, PASS)

    id_tr = "B7e755e_nbSNMPEnc60BA612C" 
             #B7e755e_nbModbusEnc4B4D008D
    
    
    print(f"\n Consultando ({id_tr})...")
    sensores = dce.consultar_equipo(id_tr)

    if sensores:
        print(f"Se recibieron {len(sensores)} sensores.")
        print(f"{'SENSORES':<50}")
        print("-" * 60)
        
        for s in sensores:
            label = s.get('label', 'N/A')
            valor = s.get('value', 'N/A')
            unidad = s.get('units', '')
            kind = s.get('kind', '')
            
            print(f"{label} {valor} {kind}")


 Consultando (B7e755e_nbSNMPEnc60BA612C)...
Renovando Token de acceso...
Token renovado exitosamente.
Se recibieron 12 sensores.
SENSORES                                          
------------------------------------------------------------
ANALOG INPUT - ML CURRENT AC S 70.50 A AMP_DETECTOR
ANALOG INPUT - ML CURRENT AC T 82.10 A AMP_DETECTOR
ANALOG INPUT - °C SALA S01 24.00 ° C TEMPERATURE
ANALOG INPUT - ML VOLTAGE AC R-S 213.86 V VOLTAGE
ANALOG INPUT - ML VOLTAGE AC S-T 213.30 V VOLTAGE
08 - CORRIENTE DC DE LAS BATERIAS 0.00 A AMP_DETECTOR
Link Status Online COMMUNICATION_SENSOR
ANALOG INPUT - ML VOLTAGE AC T-R 218.78 V VOLTAGE
ANALOG INPUT - ML CURRENT AC R 70.39 A AMP_DETECTOR
DIGITAL OUTPUT - ML AC FAIL INACTIVO STATE
ANALOG INPUT - °C SALA S02 21.00 ° C TEMPERATURE
13 - TEST DE BATERIAS DESHABILITADO STATE
